In [608]:
#Functions and Imports 

import pandas as pd
import duckdb
import openpyxl
from openpyxl.utils import get_column_letter

# Setting up SQL syntax in Python 
#conn = duckdb.connect("SampleDB",read_only=False)

# Ensure pandas ExcelWriter uses a different engine if openpyxl is not available
try:
    from openpyxl import Workbook
except ModuleNotFoundError:
    
    excel_engine = 'xlsxwriter'  
else:
    excel_engine = 'openpyxl'

#Function that checks if newfile csv has the correct datatypes to run 
#Parameters: dataframe 
#Returns: adjusted dataframe with float dtype columns
def dataType(df):
    dft = df.iloc[:,infoColumn:numEndColumn]
    dft = pd.DataFrame(dft)
    for i in range(len(dft.columns)):
        if dft[dft.columns[i]].dtype != float:
            #print(dft.columns[i])
            df[dft.columns[i]] = dft[dft.columns[i]].str.replace(',','').astype(float)
    return df    




#Variables for functions (Update if columns were added to UPS file)
infoColumn = 33
numEndColumn = 65
billedWeightColmn = 28

    

In [610]:
#importing two files: Res1 and ups csv

#importing 
UPSCSV = pd.read_csv("Invoice_00000399CF_110125.csv", encoding='latin',low_memory=False, header=None)
UPSCSV = UPSCSV.fillna('')
res1 = pd.read_excel('res1 adjusted 1.xlsx')

#Searching common tracking numbers in UPSCSV file 
UPSCSV_Res1 = UPSCSV[UPSCSV[13].isin(res1['Tracking Number'])]

#displaying dataframe
#UPSCSV_Res1


In [599]:
#Total cost validation

#Simplifying ups datafram 
UPSCSV_Res1 = pd.DataFrame(UPSCSV_Res1.iloc[:,[20,43,51,52]])

#Getting Charge Value column and adding it as new column 
sumTotal = UPSCSV_Res1.iloc[:,2] + UPSCSV_Res1.iloc[:,3]
UPSCSV_Res1['Charge Amount'] = sumTotal


In [602]:
#getting grouped by tracking number total charge  
groupedRes1 = UPSCSV_Res1.groupby(20).sum()
groupedRes1 = pd.DataFrame(groupedRes1)

#Getting the cost value for each shipment in Res1 file
res2 = pd.DataFrame(res1.iloc[:,[4]+list(range(infoColumn,numEndColumn))])

#adjusting incentive columns to be negative values
for i in range(len(res2.columns)-1):
    if (i+1)%2 == 0:
        res2.iloc[:,i+1] = res2.iloc[:,i+1] * -1
#Creating a new column to validate charges in UPS dataframe and res 
res2["Total Amount"] = res2.iloc[:,1:len(res2.columns)].sum(axis=1)

        
        

In [604]:
res2[['Tracking Number','Total Amount']]

,Tracking Number,Total Amount
0,1Z1FV6990395559755,90.11
1,1Z1FV6990396375577,80.12
2,1Z1FV6990396738961,76.60
3,1Z4X459A4204799741,31.52
4,1Z4X459A4210604322,24.33
5,1Z4X459A4215211336,24.81
6,1Z7EW7860230845078,19.79
7,1Z84W64A0308207152,14.48
8,1Z84W64A0325902547,9.65
9,1Z84W64A0333588659,9.75


In [606]:
groupedRes1

,43,51,52,Charge Amount
20,,,,
1Z1FV6990395559755,FRTACCACCFSCFRTACCFSC,142.82,90.11,232.93
1Z1FV6990396375577,FRTACCACCFSCFRTACCFSC,120.68,80.12,200.8
1Z1FV6990396738961,FRTACCACCFSCFRTACCFSC,112.9,76.6,189.5
1Z4X459A4204799741,FRTACCFSCINFFRTACCACCACCACCFSC,50.96,31.52,82.48
1Z4X459A4210604322,FRTACCFSCINFFRTACCACCACCACCFSCFRTFSC,35.0,24.33,59.33
1Z4X459A4215211336,FRTACCFSCINFFRTACCACCACCACCFSCFRTFSC,36.09,24.81,60.9
1Z7EW7860230845078,FRTACCFSCINFFRTACCACCACCFSC,64.06,19.79,83.85
1Z84W64A0308207152,FRTACCFSCINFFRTACCACCACCFSC,20.09,14.48,34.57
1Z84W64A0325902547,FRTFSCINFFRTACCFSC,16.17,9.65,25.82


In [392]:
UPSCSV_Res1

,13,43,51,52,Charge Amount
303,1ZA83B940316258936,FRT,14.38,6.11,20.49
304,1ZA83B940316258936,FSC,3.24,0.86,4.1
306,1ZA83B940316258936,INF,0.0,0.0,0.0
351,1ZA83B940316258936,FRT,-0.82,0.82,0.0
352,1ZA83B940316258936,ACC,3.9,2.2,6.1
...,...,...,...,...,...
18472,1Z84W64A0308207152,FSC,0.24,0.57,0.81
18473,1Z84W64A0333588659,FRT,-0.25,0.25,0.0
18474,1Z84W64A0333588659,ACC,3.9,2.2,6.1
18475,1Z84W64A0333588659,ACC,0.3,0.1,0.4


In [394]:
UPSCSV_Res1.iloc[:,3]

303      6.11
304      0.86
306       0.0
351      0.82
352       2.2
         ... 
18472    0.57
18473    0.25
18474     2.2
18475     0.1
18476    0.36
Name: 52, Length: 189, dtype: object

In [396]:
UPSCSV_Res1.iloc[:,13]

IndexError: single positional indexer is out-of-bounds

In [398]:
UPSCSV_Res1.iloc[:,43]

IndexError: single positional indexer is out-of-bounds

In [147]:
UPSCSV_Res1.iloc[:,43]

303      FRT
304      FSC
306      INF
351      FRT
352      ACC
        ... 
18472    FSC
18473    FRT
18474    ACC
18475    ACC
18476    FSC
Name: 43, Length: 189, dtype: object

In [149]:
UPSCSV_Res1.size

46116

In [151]:
UPSCSV_Res1.shape

(189, 244)

In [153]:
UPSCSV_Res1.iloc[0:3,13]

303    1ZA83B940316258936
304    1ZA83B940316258936
306    1ZA83B940316258936
Name: 13, dtype: object

In [155]:
UPSCSV_Res1[13]


303      1ZA83B940316258936
304      1ZA83B940316258936
306      1ZA83B940316258936
351      1ZA83B940316258936
352      1ZA83B940316258936
                ...        
18472    1Z84W64A0308207152
18473    1Z84W64A0333588659
18474    1Z84W64A0333588659
18475    1Z84W64A0333588659
18476    1Z84W64A0333588659
Name: 13, Length: 189, dtype: object

In [157]:
UPSCSV_Res1[51]

303      14.38
304       3.24
306        0.0
351      -0.82
352        3.9
         ...  
18472     0.24
18473    -0.25
18474      3.9
18475      0.3
18476     0.15
Name: 51, Length: 189, dtype: object

In [159]:
UPSCSV_Res1[52]

303      6.11
304      0.86
306       0.0
351      0.82
352       2.2
         ... 
18472    0.57
18473    0.25
18474     2.2
18475     0.1
18476    0.36
Name: 52, Length: 189, dtype: object

In [170]:
UPSCSV_Res1

,13,43,51,52
303,1ZA83B940316258936,FRT,14.38,6.11
304,1ZA83B940316258936,FSC,3.24,0.86
306,1ZA83B940316258936,INF,0.0,0.0
351,1ZA83B940316258936,FRT,-0.82,0.82
352,1ZA83B940316258936,ACC,3.9,2.2
...,...,...,...,...
18472,1Z84W64A0308207152,FSC,0.24,0.57
18473,1Z84W64A0333588659,FRT,-0.25,0.25
18474,1Z84W64A0333588659,ACC,3.9,2.2
18475,1Z84W64A0333588659,ACC,0.3,0.1


In [400]:
res1Adjusted=res1.iloc[:,[4]+list(range(infoColumn,numEndColumn))]

In [424]:
len(res1)-1

23

In [404]:
len(res1.columns)

65

In [406]:
res1Adjusted['Base Incentive Charge'] = res1Adjusted['Base Incentive Charge'] * -1

/var/folders/9p/pz_xr3bx1jq3kdjgzw17znjh0000gn/T/ipykernel_84688/179943504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res1Adjusted['Base Incentive Charge'] = res1Adjusted['Base Incentive Charge'] * -1


In [435]:
res1Adjusted

,Tracking Number,Base Freight Charge,Base Incentive Charge,Additional Handling Package,Additional Handling Package Incentive,Additional Handling Weight,Additional Handling Weight Incentive,Additional Handling Dimensions,Additional Handling Dimensions Incentive,Adult Sign Req Freight,...,Address Correction,Address Correction Incentive,Print Labels,Print Labels Incentive,Electronic Labels,Electronic Labels Incentive,Large Package Surcharge,Large Package Surcharge Incentive,Other Freight Charge,Other Incentive Charge
0,1Z1FV6990395559755,124.18,-88.17,0,0,55,27.5,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,2.15,-3.90
1,1Z1FV6990396375577,97.63,-69.32,0,0,55,27.5,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,2.15,-3.90
2,1Z1FV6990396738961,88.30,-62.69,0,0,55,27.5,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,2.15,-3.90
3,1Z4X459A4204799741,55.01,-39.06,0,0,0,0.0,0,0,11.52,...,0.00,0.00,0,0,0,0,0,0,-5.70,-3.60
4,1Z4X459A4210604322,32.06,-22.76,0,0,0,0.0,0,0,11.52,...,0.00,0.00,0,0,0,0,0,0,-1.89,-1.05
5,1Z4X459A4215211336,32.06,-22.76,0,0,0,0.0,0,0,11.52,...,0.00,0.00,0,0,0,0,0,0,-0.58,-0.17
6,1Z7EW7860230845078,63.34,-49.41,0,0,0,0.0,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,-3.79,-0.79
7,1Z84W64A0308207152,14.91,-8.91,0,0,0,0.0,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,-5.70,-3.60
8,1Z84W64A0325902547,15.98,-9.98,0,0,0,0.0,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,-6.10,-3.90
9,1Z84W64A0333588659,18.28,-12.28,0,0,0,0.0,0,0,0.00,...,0.00,0.00,0,0,0,0,0,0,-5.70,-3.60


In [441]:
res1Adjusted.iloc[:,1] = res1Adjusted.iloc[:,1] * -1

In [459]:
res1Adjusted = pd.DataFrame(res1Adjusted)

In [481]:
res1Adjusted['Total Row Value']=res1Adjusted.iloc[:,1:len(res1Adjusted.columns)].sum(axis=1)

In [463]:
res1Adjusted.iloc[:,1]

0    -124.18
1     -97.63
2     -88.30
3     -55.01
4     -32.06
5     -32.06
6     -63.34
7     -14.91
8     -15.98
9     -18.28
10    -20.49
11    -93.44
12    -18.95
13    -18.95
14    -18.95
15    -18.95
16    -36.11
17    -23.64
18    -45.54
19    -26.97
20    -19.85
21    -18.95
22    -32.02
23    -17.34
Name: Base Freight Charge, dtype: float64

In [475]:
len(res1Adjusted.columns)

33

In [483]:
res1Adjusted

,Tracking Number,Base Freight Charge,Base Incentive Charge,Additional Handling Package,Additional Handling Package Incentive,Additional Handling Weight,Additional Handling Weight Incentive,Additional Handling Dimensions,Additional Handling Dimensions Incentive,Adult Sign Req Freight,...,Address Correction Incentive,Print Labels,Print Labels Incentive,Electronic Labels,Electronic Labels Incentive,Large Package Surcharge,Large Package Surcharge Incentive,Other Freight Charge,Other Incentive Charge,Total Row Value
0,1Z1FV6990395559755,-124.18,-88.17,0,0,55,27.5,0,0,0.00,...,0.00,0,0,0,0,0,0,2.15,-3.90,-29.95
1,1Z1FV6990396375577,-97.63,-69.32,0,0,55,27.5,0,0,0.00,...,0.00,0,0,0,0,0,0,2.15,-3.90,2.48
2,1Z1FV6990396738961,-88.30,-62.69,0,0,55,27.5,0,0,0.00,...,0.00,0,0,0,0,0,0,2.15,-3.90,13.84
3,1Z4X459A4204799741,-55.01,-39.06,0,0,0,0.0,0,0,11.52,...,0.00,0,0,0,0,0,0,-5.70,-3.60,-38.06
4,1Z4X459A4210604322,-32.06,-22.76,0,0,0,0.0,0,0,11.52,...,0.00,0,0,0,0,0,0,-1.89,-1.05,-3.41
5,1Z4X459A4215211336,-32.06,-22.76,0,0,0,0.0,0,0,11.52,...,0.00,0,0,0,0,0,0,-0.58,-0.17,-1.17
6,1Z7EW7860230845078,-63.34,-49.41,0,0,0,0.0,0,0,0.00,...,0.00,0,0,0,0,0,0,-3.79,-0.79,-74.65
7,1Z84W64A0308207152,-14.91,-8.91,0,0,0,0.0,0,0,0.00,...,0.00,0,0,0,0,0,0,-5.70,-3.60,7.54
8,1Z84W64A0325902547,-15.98,-9.98,0,0,0,0.0,0,0,0.00,...,0.00,0,0,0,0,0,0,-6.10,-3.90,-9.23
9,1Z84W64A0333588659,-18.28,-12.28,0,0,0,0.0,0,0,0.00,...,0.00,0,0,0,0,0,0,-5.70,-3.60,-12.27
